# Asymptotic Iteration Method (AIM): Poschl-Teller Potential

The Poschl-Teller potential is given by: \
\
$ V(x) = \dfrac{1}{2}\,\text{sech}^2(x) = \dfrac{2}{(e^{-x}+e^x)^2} $

# Symbolic Calculations

We begin our analysis with the Schrödinger equation: \
\
$ \dfrac{d^2\psi}{dx^2} + \left( \omega^2 - \dfrac{1}{2}\text{sech}^2(x) \right)\psi = 0 $

In [2]:
from sympy import *

x,y = symbols("x y", real=True)
w = symbols("\omega")

psi = Function("\psi")(x)
psi_y = Function("\psi")(y)
phi = Function("\phi")(y)

sch_eq = Eq( diff(diff(psi,x),x) + (w**2-S(1)/2*sech(x)**2)*psi, 0 )
display(sch_eq)

Eq((\omega**2 - sech(x)**2/2)*\psi(x) + Derivative(\psi(x), (x, 2)), 0)

It is convenient to write it in terms of a bounded variable. For this, we perform the change of variables $y = \tanh(x)$.

In [3]:
#Function to change variables (https://stackoverflow.com/questions/57840957/differential-equation-change-of-variables-with-sympy)
def variable_change(ODE,dependent_var, 
                    independent_var,
                    new_dependent_var = None, 
                    new_independent_var= None, 
                    dependent_var_relation = None,
                    independent_var_relation = None,
                    order = 2):

    if new_dependent_var == None:
        new_dependent_var = dependent_var
    if new_independent_var == None:
        new_independent_var = independent_var

    # dependent variable change

    if new_independent_var != independent_var:

        for i in range(order, -1, -1):

            # remplace derivate
            a = diff(dependent_var , independent_var, i )
            ξ = Function("ξ")(independent_var)

            b = diff( dependent_var.subs(independent_var, ξ),  independent_var  ,i)

            rel = solve(independent_var_relation, new_independent_var)[0]

            for j in range(order, 0, -1):
                b = b.subs( diff(ξ,independent_var,j), diff(rel,independent_var,j))

            b = b.subs(ξ, new_independent_var)

            rel = solve(independent_var_relation, independent_var)[0]
            b = b.subs(independent_var, rel)

            ODE =   ODE.subs(a,b)

        ODE = ODE.subs(independent_var, rel)

    # change of variables of indpendent variable

    if new_dependent_var != dependent_var:

        ODE = (ODE.subs(dependent_var.subs(independent_var,new_independent_var) , (solve(dependent_var_relation, dependent_var)[0])))
        ODE = ODE.doit().expand()

    return ODE.simplify()

In [4]:
sch_eq_y = variable_change(
    ODE=sch_eq,
    independent_var=x,
    new_independent_var=y,
    independent_var_relation=Eq(x,atanh(y)),
    dependent_var=psi,
    new_dependent_var=psi,
    dependent_var_relation=None,
    order=2
)

display(sch_eq_y)

Eq(2*y*(y**2 - 1)*Derivative(\psi(y), y) + (y**2 - 1)**2*Derivative(\psi(y), (y, 2)) + (2*\omega**2 + y**2 - 1)*\psi(y)/2, 0)

We then write the eigenfunction in a form as to include the boundary conditions, $\psi = (1-y)^{-i\omega/2}(1+y)^{-i\omega/2}\phi$

In [5]:
sch_eq_y = sch_eq_y.subs(psi_y,(1-y)**(-I*w/2)*(1+y)**(-I*w/2)*phi).doit()
display(sch_eq_y)

Eq(2*y*(y**2 - 1)*(-I*\omega*\phi(y)/(2*(1 - y)**(I*\omega/2)*(y + 1)*(y + 1)**(I*\omega/2)) + I*\omega*\phi(y)/(2*(1 - y)*(1 - y)**(I*\omega/2)*(y + 1)**(I*\omega/2)) + Derivative(\phi(y), y)/((1 - y)**(I*\omega/2)*(y + 1)**(I*\omega/2))) + (y**2 - 1)**2*(-\omega**2*\phi(y)/(2*(y - 1)*(y + 1)) - \omega*(\omega - 2*I)*\phi(y)/(4*(y + 1)**2) - \omega*(\omega - 2*I)*\phi(y)/(4*(y - 1)**2) - I*\omega*Derivative(\phi(y), y)/(y + 1) - I*\omega*Derivative(\phi(y), y)/(y - 1) + Derivative(\phi(y), (y, 2)))/((1 - y)**(I*\omega/2)*(y + 1)**(I*\omega/2)) + (2*\omega**2 + y**2 - 1)*\phi(y)/(2*(1 - y)**(I*\omega/2)*(y + 1)**(I*\omega/2)), 0)

Simplifying the equation:

In [6]:
A,B,C = symbols("A B C") #A,B,C represent second, first and zero-th order derivatives of phi
sch_eq_y = sch_eq_y.subs( diff(diff(phi,y),y), A ).subs( diff(phi,y), B ).subs(phi, C)
sch_eq_y = factor(sch_eq_y)
display(sch_eq_y)

Eq((y - 1)*(y + 1)*(2*A*y**2 - 2*A - 4*I*B*\omega*y + 4*B*y - 2*C*\omega**2 - 2*I*C*\omega + C)/(2*(1 - y)**(I*\omega/2)*(y + 1)**(I*\omega/2)), 0)

In [7]:
sch_eq_y = Eq(sch_eq_y.lhs.collect(A).collect(B).collect(C),0) #Collect terms
display(sch_eq_y)

Eq((y - 1)*(y + 1)*(A*(2*y**2 - 2) + B*(-4*I*\omega*y + 4*y) + C*(-2*\omega**2 - 2*I*\omega + 1))/(2*(1 - y)**(I*\omega/2)*(y + 1)**(I*\omega/2)), 0)

In [8]:
with assuming(Q.is_true(y != 1), Q.is_true(y != -1)):
    sch_eq_y = sch_eq_y.simplify()

display(sch_eq_y)

Eq((1 - y)**(-I*\omega/2 + 1)*(y + 1)**(-I*\omega/2 + 1)*(-A*(y**2 - 1) + 2*B*y*(I*\omega - 1) + C*(2*\omega**2 + 2*I*\omega - 1)/2), 0)

In [9]:
with assuming(Q.is_true( (1-y)**(-I*w/2 + 1) != 0 ), Q.is_true( (y+1)**(-I*w/2+1) != 0 )):
    sch_eq_y = sch_eq_y.simplify().doit()

display(sch_eq_y)

Eq((1 - y)**(-I*\omega/2 + 1)*(y + 1)**(-I*\omega/2 + 1)*(-A*(y**2 - 1) + 2*B*y*(I*\omega - 1) + C*(2*\omega**2 + 2*I*\omega - 1)/2), 0)

In [10]:
#Hard-coded divide by common factor
sch_eq_y = Eq( (sch_eq_y.lhs * (1-y)**(I*w/2 - 1) * (1+y)**(I*w/2 - 1) ).simplify() , 0)
display(sch_eq_y)

Eq(-A*(y**2 - 1) + 2*B*y*(I*\omega - 1) + C*(2*\omega**2 + 2*I*\omega - 1)/2, 0)

In [11]:
#We have the same expression as in the paper
sch_eq_y = Eq( 2*B*y*(I*w-1)/(y**2-1) + C*(2*w**2+w*I*w - 1)/(2*(y**2-1)), A ).subs(A, diff(diff(phi,y),y)).subs(B, diff(phi,y)).subs(C,phi)
display(sch_eq_y)

Eq(2*y*(I*\omega - 1)*Derivative(\phi(y), y)/(y**2 - 1) + (2*\omega**2 + I*\omega**2 - 1)*\phi(y)/(2*y**2 - 2), Derivative(\phi(y), (y, 2)))

Schrödinger-like equation after change of coordinates

$ \dfrac{d^2\psi}{dx^2} + \left( \omega^2 - \dfrac{1}{2}\text{sech}^2(x) \right)\psi = 0 \implies  \dfrac{d^2\phi}{dy^2} = \dfrac{2y(1-iw)}{1-y^2}\dfrac{d\phi}{dy} + \dfrac{1-2iw-2w^2}{2(1-y^2)}\phi $

where $y = \text{tanh}(x)$ and $\psi = (1-y)^{-i\omega/2}(1+y)^{-i\omega/2}\phi$ (which fulfills the necessary boundary conditions)

# AIM algorithm

### AIM Parameters

$ \lambda_0 = \dfrac{2y(1-iw)}{1-y^2} \qquad s_0 = \dfrac{1-2iw-2w^2}{2(1-y^2)} $

### Recursion relations
$ \lambda_n = \lambda'_{n-1} + s_{n-1} + \lambda_0\lambda_{n-1} \qquad s_n = s'_{n-1} + s_0\lambda_{n-1} $

### Quantization condition

$ \delta_n = s_n\lambda_{n-1} - s_{n-1}\lambda_n $

Import necessary libraries

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import *

N = 1000

y = symbols("y", real=True)
w = symbols("\omega")

l = np.empty(N,dtype=object)
lp = np.empty(N,dtype=object)
s = np.empty(N,dtype=object)
sp = np.empty(N,dtype=object)

d = np.empty(N,dtype=object)

l[0] = (2*y*(1-I*w))/(1-y**2)
s[0] = (1-2*I*w-2*w**2)/(2*(1-y**2))

Next, compute first derivatives of lambda and s parameters

In [13]:
lp[0] = diff(l[0],y)
sp[0] = diff(s[0],y)

In [14]:
lp[0]

4*y**2*(-I*\omega + 1)/(1 - y**2)**2 + 2*(-I*\omega + 1)/(1 - y**2)

In [15]:
sp[0]

4*y*(-2*\omega**2 - 2*I*\omega + 1)/(2 - 2*y**2)**2

Calculate n-th (1st) values of lambda and s parameter

In [16]:
l[1] = lp[0]+s[0]+l[0]*l[0]
s[1] = sp[0] + s[0]*l[0]

Evaluate the quantization condition $\delta_n$ at point $y = 0$ obtaining polynomial in $\omega$

In [17]:
d[1] = s[1]*l[0] - s[0]*l[1]
#d[1].subs(y,0)
p = simplify(d[1].subs(y,0))
d[1]


2*y*(-I*\omega + 1)*(4*y*(-2*\omega**2 - 2*I*\omega + 1)/(2 - 2*y**2)**2 + 2*y*(-I*\omega + 1)*(-2*\omega**2 - 2*I*\omega + 1)/((1 - y**2)*(2 - 2*y**2)))/(1 - y**2) - (-2*\omega**2 - 2*I*\omega + 1)*(4*y**2*(-I*\omega + 1)**2/(1 - y**2)**2 + 4*y**2*(-I*\omega + 1)/(1 - y**2)**2 + (-2*\omega**2 - 2*I*\omega + 1)/(2 - 2*y**2) + 2*(-I*\omega + 1)/(1 - y**2))/(2 - 2*y**2)

In [18]:
p

-\omega**4 - 4*I*\omega**3 + 6*\omega**2 + 4*I*\omega - 5/4

Solve the polynomial for the solution of $\omega$, taking only those with negative imaginary part and positive real part

In [19]:
sols = solve(p)
print("All solutions: " + str(sols))
print("Filtered solutions:")
for i in range(len(sols)):
    if re(sols[i]) > 0 and  im(sols[i]) < 0:
        print("w_" + str(i+1) + " = " + str(sols[i]))

All solutions: [-1/2 - 3*I/2, -1/2 - I/2, 1/2 - 3*I/2, 1/2 - I/2]
Filtered solutions:
w_3 = 1/2 - 3*I/2
w_4 = 1/2 - I/2


### Second Iteration

In [20]:
lp[1] = diff(l[1],y)
sp[1] = diff(s[1],y)
l[2] = lp[1] + s[1] + l[0]*l[1]
s[2] = sp[1] + s[0]*l[1]

In [21]:
d[2] = s[2]*l[1] - s[1]*l[2]
d[2]

-(4*y*(-2*\omega**2 - 2*I*\omega + 1)/(2 - 2*y**2)**2 + 2*y*(-I*\omega + 1)*(-2*\omega**2 - 2*I*\omega + 1)/((1 - y**2)*(2 - 2*y**2)))*(16*y**3*(-I*\omega + 1)**2/(1 - y**2)**3 + 16*y**3*(-I*\omega + 1)/(1 - y**2)**3 + 8*y*(-2*\omega**2 - 2*I*\omega + 1)/(2 - 2*y**2)**2 + 2*y*(-I*\omega + 1)*(4*y**2*(-I*\omega + 1)**2/(1 - y**2)**2 + 4*y**2*(-I*\omega + 1)/(1 - y**2)**2 + (-2*\omega**2 - 2*I*\omega + 1)/(2 - 2*y**2) + 2*(-I*\omega + 1)/(1 - y**2))/(1 - y**2) + 2*y*(-I*\omega + 1)*(-2*\omega**2 - 2*I*\omega + 1)/((1 - y**2)*(2 - 2*y**2)) + 8*y*(-I*\omega + 1)**2/(1 - y**2)**2 + 12*y*(-I*\omega + 1)/(1 - y**2)**2) + (4*y**2*(-I*\omega + 1)**2/(1 - y**2)**2 + 4*y**2*(-I*\omega + 1)/(1 - y**2)**2 + (-2*\omega**2 - 2*I*\omega + 1)/(2 - 2*y**2) + 2*(-I*\omega + 1)/(1 - y**2))*(32*y**2*(-2*\omega**2 - 2*I*\omega + 1)/(2 - 2*y**2)**3 + 8*y**2*(-I*\omega + 1)*(-2*\omega**2 - 2*I*\omega + 1)/((1 - y**2)*(2 - 2*y**2)**2) + 4*y**2*(-I*\omega + 1)*(-2*\omega**2 - 2*I*\omega + 1)/((1 - y**2)**2*(2 -

In [22]:
p = simplify(d[2].subs(y,0))
p

-\omega**6 - 9*I*\omega**5 + 65*\omega**4/2 + 60*I*\omega**3 - 241*\omega**2/4 - 129*I*\omega/4 + 65/8

In [23]:
sols = solve(p)
print("All solutions: " + str(sols))
print("Filtered solutions:")
for i in range(len(sols)):
    if re(sols[i]) > 0 and  im(sols[i]) < 0:
        print("w_" + str(i+1) + " = " + str(sols[i]))

All solutions: [-1/2 - 5*I/2, -1/2 - 3*I/2, -1/2 - I/2, 1/2 - 5*I/2, 1/2 - 3*I/2, 1/2 - I/2]
Filtered solutions:
w_4 = 1/2 - 5*I/2
w_5 = 1/2 - 3*I/2
w_6 = 1/2 - I/2


### Third iteration

In [24]:
lp[2] = diff(l[2],y)
sp[2] = diff(s[2],y)
l[3] = lp[2] + s[2] + l[0]*l[2]
s[3] = sp[2] + s[0]*l[2]
d[3] = s[3]*l[2] - s[2]*l[3]
p = simplify(d[3].subs(y,0))
sols = solve(p)
print("All solutions: " + str(sols))
print("Filtered solutions:")
for i in range(len(sols)):
    if re(sols[i]) > 0 and  im(sols[i]) < 0:
        print("w_" + str(i+1) + " = " + str(sols[i]))

All solutions: [-1/2 - 7*I/2, -1/2 - 5*I/2, -1/2 - 3*I/2, -1/2 - I/2, 1/2 - 7*I/2, 1/2 - 5*I/2, 1/2 - 3*I/2, 1/2 - I/2]
Filtered solutions:
w_5 = 1/2 - 7*I/2
w_6 = 1/2 - 5*I/2
w_7 = 1/2 - 3*I/2
w_8 = 1/2 - I/2


### Fourth iteration

In [25]:
lp[3] = diff(l[3],y)
sp[3] = diff(s[3],y)
l[4] = lp[3] + s[3] + l[0]*l[3]
s[4] = sp[3] + s[0]*l[3]
d[4] = s[4]*l[3] - s[3]*l[4]
p = simplify(d[4].subs(y,0))
sols = solve(p)
print("All solutions: " + str(sols))
print("Filtered solutions:")
for i in range(len(sols)):
    if re(sols[i]) > 0 and  im(sols[i]) < 0:
        print("w_" + str(i+1) + " = " + str(sols[i]))

All solutions: [-1/2 - 9*I/2, -1/2 - 7*I/2, -1/2 - 5*I/2, -1/2 - 3*I/2, -1/2 - I/2, 1/2 - 9*I/2, 1/2 - 7*I/2, 1/2 - 5*I/2, 1/2 - 3*I/2, 1/2 - I/2]
Filtered solutions:
w_6 = 1/2 - 9*I/2
w_7 = 1/2 - 7*I/2
w_8 = 1/2 - 5*I/2
w_9 = 1/2 - 3*I/2
w_10 = 1/2 - I/2


# Improved AIM Algorithm (IAIM)

Main idea, expand by Taylor series the coefficients $\lambda_n$ and $s_n$, evaluated at point $y = y_0$, as: \
\
$ \lambda_n = \sum_{k}  c_n^k (y-y_0)^k $ \
\
$ s_n = \sum_{k} d_n^k (y-y_0)^k  $

The coefficients follow a recursion relation (matrix elements row $i$ column $n$ of matrices $C$ and $D$): \
\
$ c_{n+1}^i = (i+1)c_{n}^{i+1} + d_{n}^i + \sum_k c_0^kc_{n}^{i-k} $ \
\
$ d_{n+1}^i = (i+1)d_{n}^{i+1} \sum_k d_0^k c_{n}^{i-k} $

The quantization condition can be written as (for the n-th iteration): \
\
$\qquad \delta_n^0= c_{n-1}^0 - d_{n-1}^0c_n^0 = 0 $

Parameters: \
\
$\qquad \lambda_0 = \dfrac{2y(1-iw)}{1-y^2} \qquad s_0 = \dfrac{1-2iw-2w^2}{2(1-y^2)} $

In [185]:
import numpy as np
from sympy import *

y = symbols("y", real=True)
w = symbols("\omega")

N = 20 #NUMBER OF ITERATIONS TO PERFORM

#INITIAL PARAMETERS
l0 = (2*y*(1-I*w))/(1-y**2)
s0 = (1-2*I*w-2*w**2)/(2*(1-y**2))

We must compute the Taylor series expansion of the parameters $\lambda_0$ and $s_0$ up to the term $N$

In [186]:
#FUNCTION THAT GETS COEFFICIENTS AS LIST FROM SERIES EXPANSION
def get_coeff(a):
    
    coeff = []
    coeff = np.append(coeff, a.subs(y,0))
    for i in range(1,N):
        coeff = np.append(coeff, a.coeff(y**i))

    return coeff

In [187]:
l0_series = series(l0,y,0,N).removeO()
display(l0_series.collect(y))
s0_series = series(s0,y,0,N).removeO()
display(s0_series.collect(y))
l0_coeff = get_coeff(l0_series)
s0_coeff = get_coeff(s0_series)

y**19*(-2*I*\omega + 2) + y**17*(-2*I*\omega + 2) + y**15*(-2*I*\omega + 2) + y**13*(-2*I*\omega + 2) + y**11*(-2*I*\omega + 2) + y**9*(-2*I*\omega + 2) + y**7*(-2*I*\omega + 2) + y**5*(-2*I*\omega + 2) + y**3*(-2*I*\omega + 2) + y*(-2*I*\omega + 2)

-\omega**2 - I*\omega + y**18*(-\omega**2 - I*\omega + 1/2) + y**16*(-\omega**2 - I*\omega + 1/2) + y**14*(-\omega**2 - I*\omega + 1/2) + y**12*(-\omega**2 - I*\omega + 1/2) + y**10*(-\omega**2 - I*\omega + 1/2) + y**8*(-\omega**2 - I*\omega + 1/2) + y**6*(-\omega**2 - I*\omega + 1/2) + y**4*(-\omega**2 - I*\omega + 1/2) + y**2*(-\omega**2 - I*\omega + 1/2) + 1/2

The coefficients of these series expansions are columns of the matrices $C$ and $D$, with $i$ index representing the rows of the column, and $N$ (iteration number) representing the column. We begin by defining the first column of $C$ ($D$) as the coefficients of the Taylor expansion of $\lambda_0$ ($s_0$).

In [188]:
C = np.zeros((N,N+1),dtype=object)
D = np.zeros((N,N+1),dtype=object)

#Initial parameters
for i in range(N):
    C[i,0] = l0_coeff[i]
for i in range(N):
    D[i,0] = s0_coeff[i]

In [189]:
#display((Matrix(C)))
#display((Matrix(D)))

Now, we can calculate the next columns of the matrix using the recursion equations given previously.

In [190]:
for n in range(0,N-1): #COLUMNS
    for i in range(0,N): #ROWS
        if (i+1 == N):
            D[i,n+1] = 0
            C[i,n+1] = D[i,n]
        else:
            D[i,n+1] = (i+1)*D[i+1,n]
            C[i,n+1] = (i+1)*C[i+1,n]+D[i,n]
        for k in range(0,i+1): #SUM INTERIOR
            C[i,n+1] = C[i,n+1] + C[k,0]*C[i-k,n]
            D[i,n+1] = D[i,n+1] + D[k,0]*C[i-k,n]

In [191]:
#Blows up for big N
#display(Matrix(C))
#display(Matrix(D))

Once we have found both matrices, we can apply the quantization condition for the two first rows of each column and each matrix to obtain the polynomial we must solve in $\omega$.

In [193]:
d = D[0,n]*C[0,n-1] - D[0,n-1]*C[0,n]
d.expand()

KeyboardInterrupt: 

In [ ]:
sols = solve(d,w)
print("All solutions:")
print(sols)
print("Filtered solutions:")
for i in range(len(sols)):
    if (re(sols[i]) > 0 and  im(sols[i]) < 0):
        print("w_" + str(i+1) + " = " + str(sols[i]))

KeyboardInterrupt: 